In [1]:
import os

In [2]:
%pwd

'c:\\WORK\\Projects\\MLOps\\Project1\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\WORK\\Projects\\MLOps\\Project1'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: Path
    alpha: str
    l1_ratio: str
    target_column: str
        


In [7]:
from src.Project1.constants import *
from src.Project1.utils.common import read_yaml, create_dirs

In [8]:
class ConifguraioManager:
    def __init__(self,
                 config_filepath = config_file_path,
                 params_filepath = params_file_path,
                 schema_filepath = schema_file_path):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        #create_dirs([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_dirs([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )
        return model_trainer_config

In [9]:
import pandas as pd
from sklearn.linear_model import ElasticNet
import os 
from src.Project1 import logger
import joblib

In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config 
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_X = train_data.drop(self.config.target_column, axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        test_X = test_data.drop([self.config.target_column], axis=1)

        lr = ElasticNet(alpha= self.config.alpha, l1_ratio= self.config.l1_ratio, random_state=42)
        lr.fit(train_X , train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [11]:
try:
    config = ConifguraioManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-04-06 12:42:25,875] {c:\WORK\Projects\MLOps\Project1\src\Project1\utils\common.py:31} INFO -  Read yaml file from config\config.yaml
[2025-04-06 12:42:25,879] {c:\WORK\Projects\MLOps\Project1\src\Project1\utils\common.py:31} INFO -  Read yaml file from params.yaml
[2025-04-06 12:42:25,885] {c:\WORK\Projects\MLOps\Project1\src\Project1\utils\common.py:31} INFO -  Read yaml file from schema.yaml
[2025-04-06 12:42:25,888] {c:\WORK\Projects\MLOps\Project1\src\Project1\utils\common.py:51} INFO - Created directory artifacts/model_trainer
